In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder, DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests

import json
import pickle

import csv
import os
import time
from pprint import pprint

In [22]:
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "berlin": {"iconservice": "https://berlin.net.solidwallet.io", "nid": 7},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}}

network = "mainnet"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
env = connections[network]
NID=connections["berlin"]["nid"]
GOVERNANCE_ADDRESS ="cx0000000000000000000000000000000000000000"

In [8]:
print(connections["berlin"])

{'iconservice': 'https://berlin.net.solidwallet.io', 'nid': 7}


In [30]:
icon_service= IconService(HTTPProvider(connections["berlin"]["iconservice"],3))


In [12]:

@retry(JSONRPCException, tries=10, delay=1, back_off=1)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


In [25]:
# cell 4
# initialized all methods
def get_status_and_print(to, method, response) -> int:
    """
    get status and print response
    :param to:
    :param method:
    :param response:
    :return: status
    """
    status = response['status']
    print(f'SCORE {to} ::: Method {method} ::: Status: {status}')
    if len(response["eventLogs"]) > 0:
        for item in response["eventLogs"]:
            print(f'{item} \n')
    if status == 0:
        print(f'Failure: {response["failure"]}')
    return status


def _deploy_contract(_contract_name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_contract_name)) \
        .params(params) \
        .build()
    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)
    res = get_tx_result(tx_hash)
    status = get_status_and_print(_contract_name, "deploy", res)

    return {
        "address": res.get('scoreAddress', ''),
        "status": status
    }


def deploy_contract(_contract_name,params):
    """
    deploy all contracts excepts oToken and dToken
    :param _contract_name:  name of contract - should match with package name
    :return: SCORE address
    """
    result = _deploy_contract(_contract_name,params)
    print("deploy_contract -> ",result["status"])
    return result["address"]

def deposit_transaction(_from, _to, _value, _method="add"):
    transaction = DepositTransactionBuilder() \
        .from_(_from.get_address()) \
        .to(contracts[_to]) \
        .nid(3) \
        .value(_value) \
        .step_limit(1000000000) \
        .nonce(100) \
        .action(_method) \
        .build()
    
    signed_transaction = SignedTransaction(transaction, _from)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status





def send_tx(_to, _method, _params, _value, _wallet) -> int:
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    print(f'Calling {_method}, with parameters {_params} on the {_to} contract.')
    transaction = CallTransactionBuilder() \
        .from_(_wallet.get_address()) \
        .to(contracts[_to]) \
        .value(_value) \
        .step_limit(30000000) \
        .nid(NID) \
        .nonce(100) \
        .method(_method) \
        .params(_params) \
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status,response
def readOnlyCall(_to, _method, _params, _wallet):
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    _call = CallBuilder() \
        .from_(_wallet.get_address()) \
        .to(contracts[_to]) \
        .method(_method) \
        .params(_params) \
        .build()
    result = icon_service.call(_call)
    return result

In [13]:
private="eef769ccffeda60de514dc5e41c3d1c354efcc6ae58d8406ee03ad06b188e5b3"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()

'hxad4edfdb6d5834a4350a74bea5448535385c07d5'

In [21]:
a="cx087b4164a87fdfb7b714f3bafe9dfb050fd6b132"
len(a)
b = "cx"+ "0"*40
print(b,len(b))

cx0000000000000000000000000000000000000000 42


In [26]:
basic =deploy_contract("std_reference_basic",{})

Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending: Pending'}


{'code': -31002, 'message': 'Pending: Pending'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...
SCORE std_reference_basic ::: Method deploy ::: Status: 1
deploy_contract ->  1


'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c'

In [28]:
proxy = deploy_contract("std_reference_proxy",{"_ref":basic})

Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending: Pending'}


{'code': -31002, 'message': 'Pending: Pending'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...
SCORE std_reference_proxy ::: Method deploy ::: Status: 1
deploy_contract ->  1


In [29]:
print(proxy)

cx36a55b74aca43a9db9d5a8fc876c76d04daa85a2
